<a href="https://colab.research.google.com/github/Lakshmisys/Twitter-sentiment-analysis/blob/main/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import nltk

import re
import string
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_csv('Twitter_Data.csv')

In [ ]:
df.shape

(16090, 2)

In [ ]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,Negative
1,talk all the nonsense and continue all the dra...,Neutral
2,what did just say vote for modi welcome bjp t...,Positive
3,asking his supporters prefix chowkidar their n...,Positive
4,answer who among these the most powerful world...,Positive


In [ ]:
df.category.value_counts()

 1.0    6675
 0.0    5572
-1.0    3842
Name: category, dtype: int64

In [ ]:
df['category']=df['category'].map({-1.0:'Negative', 0.0:'Neutral', 1.0:'Positive'})

In [ ]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,Negative
1,talk all the nonsense and continue all the dra...,Neutral
2,what did just say vote for modi welcome bjp t...,Positive
3,asking his supporters prefix chowkidar their n...,Positive
4,answer who among these the most powerful world...,Positive


In [ ]:
df.rename(columns = {'clean_text':'Tweet'}, inplace = True)
df.head()


,Tweet,category
0,when modi promised “minimum government maximum...,Negative
1,talk all the nonsense and continue all the dra...,Neutral
2,what did just say vote for modi welcome bjp t...,Positive
3,asking his supporters prefix chowkidar their n...,Positive
4,answer who among these the most powerful world...,Positive


In [ ]:
df.isna().sum()

Tweet       0
category    0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

Tweet       0
category    0
dtype: int64

In [ ]:
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
stopWords = stopwords.words('english')
stopWords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

In [ ]:
def cleanData(text):

    # To convert the all uppercase to lowercase
    text = text.lower()

    # This is a reguglar expression to replace anything char that is not alphabet or numeric.
    text = re.sub(r"[^A-Za-z0-9]",' ', text)

    # The above regular expression itself will take care of punctuation, below is an alternative to remove only punctuation.
    text = ''.join([char for char in text if char not in punct])

    # This will remove the stopwords and lemmatize the remaining word to its root word.
    text = [wn.lemmatize(word) for word in text.split(' ') if ((word not in stopWords) & len(word)!=0)]

    return ' '.join(text)

In [ ]:
df['Tweet'] = df['Tweet'].apply(cleanData)

In [ ]:
df

,Tweet,category
0,minimum maximum begin difficult job reforming ...,Negative
1,drama,Neutral
2,say welcome bjp rahul think relax,Positive
3,chowkidar name great service confusion crustal...,Positive
4,among world today trump putin may,Positive
...,...,...
16084,come necessary implemented plan big,Positive
16085,sorry see condition sabarmati road gujarat guj...,Positive
16086,feel total holding day state claimed someone t...,Positive
16087,shabana row javed mentioned mislead,Neutral


In [ ]:
df['Tweet'][0]

'minimum maximum begin difficult job reforming state year get justice state temple'

In [ ]:
def find_len(txt):
    return len(txt.split())

In [ ]:
df['Txt_len'] = [find_len(txt) for txt in df['Tweet']]

In [ ]:
df.head()

,Tweet,category,Txt_len
0,minimum maximum begin difficult job reforming ...,Negative,12
1,drama,Neutral,1
2,say welcome bjp rahul think relax,Positive,6
3,chowkidar name great service confusion crustal...,Positive,10
4,among world today trump putin may,Positive,6


In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
X = df.drop(["category","Txt_len"],axis = 1)
y = df.category

In [ ]:
messages=X.copy()

In [ ]:
#first tweet
messages['Tweet'][0]

'minimum maximum begin difficult job reforming state year get justice state temple'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['Tweet'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
11088
11089
11090
11091
11092
11093
11094
11095
11096
11097
11098
11099
11100
11101
11102
11103
11104
11105
11106
11107
11108
11109
11110
11111
11112
11113
11114
11115
11116
11117
11118
11119
11120
11121
11122
11123
11124
11125
11126
11127
11128
11129
11130
11131
11132
11133
11134
11135
11136
11137
11138
11139
11140
11141
11142
11143
11144
11145
11146
11147
11148
11149
11150
11151
11152
11153
11154
11155
11156
11157
11158
11159
11160
11161
11162
11163
11164
11165
11166
11167
11168
11169
11170
11171
11172
11173
11174
11175
11176
11177
11178
11179
11180
11181
11182
11183
11184
11185
11186
11187
11188
11189
11190
11191
11192
11193
11194
11195
11196
11197
11198
11199
11200
11201
11202
11203
11204
11205
11206
11207
11208
11209
11210
11211
11212
11213
11214
11215
11216
11217
11218
11219
11220
11221
11222
11223
11224
11225
11226
11227
11228
11229
11230
11231
11232
11233
11234
11235
11236
11237
11238
11239
11240
11241
11242
11243
11244
11245
1

In [ ]:
corpus

['minimum maximum begin difficult job reform state year get justic state templ',
 'drama',
 'say welcom bjp rahul think relax',
 'chowkidar name great servic confus crustal clear crass nonsens see',
 'among world today trump putin may',
 'tho refresh maarkefir comment',
 'surat woman perform yagna seek grace',
 'come cabinet',
 'india go import current lead elect combin look juici bear',
 'gay',
 'gst good tax upper would say muslim constitu',
 'tuthukudi would behav nationalist likli cabinet benifit',
 '',
 'one answer modi clarion extend support kar',
 'one answer modi clarion extend support',
 'parti leadershipwho bjp parti',
 'creat',
 'deserv modi elect',
 'word talk swami chowkidar tweet terror',
 'write chowkidar tri visit plz',
 'one put gen hooda support great',
 'bjp given khammam parliamentari khammam today',
 'crush shoutmodimodi jd mla',
 'sultanpur uttar pradesh candid pawan kumar given current condid popular candid bsp candid sonbhadra singh',
 'nehru aliv still aliv hea

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3371, 2557, 2915, 3547, 2281, 1387, 3511, 3026, 4777, 4351, 3511, 3162],
 [4556],
 [1981, 1654, 4986, 3364, 4624, 3811],
 [3890, 827, 3237, 3842, 2309, 1173, 1761, 3590, 2007, 3969],
 [4462, 3768, 3373, 294, 2246, 2704],
 [1018, 365, 3169, 3789],
 [3407, 4063, 782, 4290, 817, 558],
 [547, 457],
 [3190, 3109, 1041, 2763, 2708, 4960, 3916, 1912, 12, 3018],
 [3576],
 [3921, 3636, 3878, 836, 3824, 1981, 4665, 179],
 [416, 3824, 4200, 293, 3916, 457, 3275],
 [],
 [793, 4718, 532, 816, 854, 439, 2051],
 [793, 4718, 532, 816, 854, 439],
 [3237, 4998, 4986, 3237],
 [787],
 [2324, 532, 4960],
 [139, 2861, 178, 3890, 490, 4021],
 [202, 3890, 4542, 3262, 915],
 [793, 2155, 2331, 3001, 439, 3237],
 [4986, 1853, 494, 3731, 494, 3373],
 [2304, 81, 3088, 2149],
 [2734,
  1115,
  4471,
  1220,
  4957,
  1095,
  1853,
  2763,
  192,
  3021,
  1220,
  4135,
  1220,
  486,
  2272],
 [4471, 1962, 4310, 1962, 3193, 942, 2490, 4471, 869],
 [4412, 53, 3755, 793, 4572, 3240, 3747, 3190],
 [3963, 2262, 2907,

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length=20 # sentance length
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length) # add padding from front
print(embedded_docs)

[[   0    0    0 ... 4351 3511 3162]
 [   0    0    0 ...    0    0 4556]
 [   0    0    0 ... 3364 4624 3811]
 ...
 [   0    0    0 ... 4777   53  773]
 [   0    0    0 ... 4820 2284 3964]
 [   0    0    0 ... 4284 3566 1646]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0, 3371, 2557, 2915,
       3547, 2281, 1387, 3511, 3026, 4777, 4351, 3511, 3162], dtype=int32)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(3,activation='softmax'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 3)                 603       
                                                                 
Total params: 313,403
Trainable params: 313,403
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(16088, (16088,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((16088, 20), (16088,))

In [ ]:
y_final

array(['Negative', 'Neutral', 'Positive', ..., 'Positive', 'Neutral',
       'Positive'], dtype=object)

In [ ]:
y_final = pd.get_dummies(y_final)
y_final

,Negative,Neutral,Positive
0,1,0,0
1,0,1,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
16083,0,0,1
16084,0,0,1
16085,0,0,1
16086,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [ ]:
### Finally Training
model1.fit(X_train,y_train, validation_data=(X_test,y_test),epochs=10,batch_size=64)



Epoch 1/10
202/202 [==============================] - 23s 82ms/step - loss: 0.5894 - accuracy: 0.5092 - val_loss: 0.5577 - val_accuracy: 0.5463
Epoch 2/10
202/202 [==============================] - 18s 87ms/step - loss: 0.4949 - accuracy: 0.6297 - val_loss: 0.5265 - val_accuracy: 0.5948
Epoch 3/10
202/202 [==============================] - 16s 79ms/step - loss: 0.4311 - accuracy: 0.7022 - val_loss: 0.5399 - val_accuracy: 0.5864
Epoch 4/10
202/202 [==============================] - 17s 86ms/step - loss: 0.3952 - accuracy: 0.7352 - val_loss: 0.5549 - val_accuracy: 0.5768
Epoch 5/10
202/202 [==============================] - 24s 117ms/step - loss: 0.3718 - accuracy: 0.7574 - val_loss: 0.5938 - val_accuracy: 0.5938
Epoch 6/10
202/202 [==============================] - 15s 76ms/step - loss: 0.3513 - accuracy: 0.7705 - val_loss: 0.6324 - val_accuracy: 0.5848
Epoch 7/10
202/202 [==============================] - 15s 76ms/step - loss: 0.3350 - accuracy: 0.7827 - val_loss: 0.6608 - val_accuracy

In [ ]:

y_pred1 = model1.predict(X_test)

101/101 [==============================] - 2s 9ms/step


In [ ]:
y_pred1[0] #  first prediction

array([9.9940407e-01, 2.2272990e-04, 3.7299527e-04], dtype=float32)

In [ ]:
y_pred1[0] #  first prediction

array([9.9940407e-01, 2.2272990e-04, 3.7299527e-04], dtype=float32)

In [ ]:
y_pred1[0:10]

array([[9.9940407e-01, 2.2272990e-04, 3.7299527e-04],
       [8.8055225e-05, 2.4883279e-03, 9.9742359e-01],
       [1.8052863e-04, 7.4936259e-05, 9.9974453e-01],
       [1.6574936e-02, 7.6145583e-01, 2.2196935e-01],
       [1.7799504e-01, 3.1067672e-01, 5.1132822e-01],
       [1.8991615e-01, 3.2933769e-01, 4.8074606e-01],
       [2.8039530e-04, 4.3294040e-04, 9.9928665e-01],
       [9.9994749e-01, 2.0388063e-05, 3.2027547e-05],
       [3.6944099e-02, 8.9555407e-01, 6.7501806e-02],
       [3.8385529e-02, 9.2377639e-01, 3.7838038e-02]], dtype=float32)

In [ ]:
ex = y_pred1.copy()
ex[1]

array([8.8055225e-05, 2.4883279e-03, 9.9742359e-01], dtype=float32)

In [ ]:
# Maintain prediction by normalizing data
count = 0
for i in range(len(ex)):
    if (ex[i][0] > ex[i][1] and ex[i][0] > ex[i][2]):
        count = 0

    elif(ex[i][0] < ex[i][1] and ex[i][1] > ex[i][2]):
        count = 1

    elif(ex[i][0] < ex[i][2] and ex[i][1] < ex[i][2]):
        count = 2
    print(i,"for count is",count)
    for j in range(3):
        ex[i][j] = 0
        ex[i][count] = 1
    print(ex[i])
    count = 0

Streaming output truncated to the last 5000 lines.
718 for count is 0
[1. 0. 0.]
719 for count is 1
[0. 1. 0.]
720 for count is 1
[0. 1. 0.]
721 for count is 2
[0. 0. 1.]
722 for count is 2
[0. 0. 1.]
723 for count is 2
[0. 0. 1.]
724 for count is 0
[1. 0. 0.]
725 for count is 1
[0. 1. 0.]
726 for count is 1
[0. 1. 0.]
727 for count is 1
[0. 1. 0.]
728 for count is 2
[0. 0. 1.]
729 for count is 1
[0. 1. 0.]
730 for count is 2
[0. 0. 1.]
731 for count is 0
[1. 0. 0.]
732 for count is 2
[0. 0. 1.]
733 for count is 0
[1. 0. 0.]
734 for count is 2
[0. 0. 1.]
735 for count is 2
[0. 0. 1.]
736 for count is 2
[0. 0. 1.]
737 for count is 1
[0. 1. 0.]
738 for count is 1
[0. 1. 0.]
739 for count is 2
[0. 0. 1.]
740 for count is 1
[0. 1. 0.]
741 for count is 0
[1. 0. 0.]
742 for count is 0
[1. 0. 0.]
743 for count is 0
[1. 0. 0.]
744 for count is 1
[0. 1. 0.]
745 for count is 1
[0. 1. 0.]
746 for count is 1
[0. 1. 0.]
747 for count is 2
[0. 0. 1.]
748 for count is 2
[0. 0. 1.]
749 for count is 1


In [ ]:
y_test

,Negative,Neutral,Positive
8658,0,0,1
15635,0,1,0
15291,0,0,1
4269,0,1,0
9520,0,0,1
...,...,...,...
2117,0,0,1
14313,0,1,0
7831,0,1,0
7307,0,0,1


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,ex)

0.5692977004350528

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,ex))

              precision    recall  f1-score   support

           0       0.48      0.46      0.47       767
           1       0.57      0.65      0.61      1088
           2       0.63      0.57      0.59      1363

   micro avg       0.57      0.57      0.57      3218
   macro avg       0.56      0.56      0.56      3218
weighted avg       0.57      0.57      0.57      3218
 samples avg       0.57      0.57      0.57      3218



In [ ]:
!pip install keras.preprocessing.sequence-pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement keras.preprocessing.sequence-pad_sequences (from versions: none)
ERROR: No matching distribution found for keras.preprocessing.sequence-pad_sequences


In [ ]:
# Importing the libraries
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import math
from sklearn.metrics import mean_squared_error

In [ ]:
#orignal model
## LSTM model
embed_dim = 100
lstm_out = 196
max_fatures = 5000

model_lstm = Sequential()
model_lstm.add(Embedding(max_fatures, embed_dim,input_length = sent_length))
model_lstm.add(SpatialDropout1D(0.4))
model_lstm.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(3,activation='softmax'))
model_lstm.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model_lstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 100)           500000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 20, 100)          0         
 ropout1D)                                                       
                                                                 
 lstm_1 (LSTM)               (None, 196)               232848    
                                                                 
 dense_1 (Dense)             (None, 3)                 591       
                                                                 
Total params: 733,439
Trainable params: 733,439
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:

model_lstm.fit(X_train,y_train,validation_data=(X_test,y_test), epochs = 10, batch_size = 32)

Epoch 1/10
403/403 [==============================] - 59s 136ms/step - loss: 0.9546 - accuracy: 0.5428 - val_loss: 0.8832 - val_accuracy: 0.5929
Epoch 2/10
403/403 [==============================] - 57s 142ms/step - loss: 0.7888 - accuracy: 0.6591 - val_loss: 0.8714 - val_accuracy: 0.5895
Epoch 3/10
403/403 [==============================] - 60s 148ms/step - loss: 0.7144 - accuracy: 0.6974 - val_loss: 0.9159 - val_accuracy: 0.5929
Epoch 4/10
403/403 [==============================] - 56s 140ms/step - loss: 0.6611 - accuracy: 0.7214 - val_loss: 0.9515 - val_accuracy: 0.5889
Epoch 5/10
403/403 [==============================] - 57s 142ms/step - loss: 0.6109 - accuracy: 0.7500 - val_loss: 0.9995 - val_accuracy: 0.5914
Epoch 6/10
403/403 [==============================] - 56s 139ms/step - loss: 0.5707 - accuracy: 0.7698 - val_loss: 1.0124 - val_accuracy: 0.5839
Epoch 7/10
403/403 [==============================] - 55s 136ms/step - loss: 0.5306 - accuracy: 0.7832 - val_loss: 1.0892 - val_ac